In [1]:
import torch
import pandas as pd
from pydub import AudioSegment
from os import listdir
from os.path import isfile, join
from aimpathy_constants import *
from scipy import signal
from scipy.io import wavfile
import struct
from tqdm import tqdm

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

In [2]:
AUDIO_FOLDER = "C:\\Users\\amity\\PycharmProjects\\aimpathy\\data\\PMEmo\\PMEmo2019\\chorus"
THAYER_ANOTATIONS_CSV = "C:\\Users\\amity\\PycharmProjects\\aimpathy\\data\\PMEmo\\PMEmo2019\\annotations\\dynamic_annotations.csv"
thayer_annotations_df = pd.read_csv(THAYER_ANOTATIONS_CSV)

In [3]:
audio_files = [f for f in listdir(AUDIO_FOLDER) if isfile(join(AUDIO_FOLDER, f))]

In [4]:
audio_data = dict()

In [6]:
for audio_file in tqdm(audio_files, total=len(audio_files)):
    try:
        sound = AudioSegment.from_mp3(join(AUDIO_FOLDER, audio_file)).set_channels(1)
        audio_file_wave = sound.export(format="wav", bitrate=RATE)
        sample_rate, samples = wavfile.read(audio_file_wave)
        audio_data[audio_file] = samples
        audio_file_wave.close()
    except Exception as e:
        print(f"Failed reading {audio_file}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 793/793 [02:20<00:00,  5.66it/s]


In [16]:
samples_data = np.array(struct.unpack('%dh' % (len(audio_data['6.mp3'])), audio_data['6.mp3']))

In [17]:
f, t, Sxx = signal.spectrogram(samples_data, sample_rate, scaling="spectrum", nperseg=FFT_POINTS)

In [18]:
t

array([ 0.04643991,  0.12770975,  0.20897959,  0.29024943,  0.37151927,
        0.45278912,  0.53405896,  0.6153288 ,  0.69659864,  0.77786848,
        0.85913832,  0.94040816,  1.021678  ,  1.10294785,  1.18421769,
        1.26548753,  1.34675737,  1.42802721,  1.50929705,  1.59056689,
        1.67183673,  1.75310658,  1.83437642,  1.91564626,  1.9969161 ,
        2.07818594,  2.15945578,  2.24072562,  2.32199546,  2.40326531,
        2.48453515,  2.56580499,  2.64707483,  2.72834467,  2.80961451,
        2.89088435,  2.9721542 ,  3.05342404,  3.13469388,  3.21596372,
        3.29723356,  3.3785034 ,  3.45977324,  3.54104308,  3.62231293,
        3.70358277,  3.78485261,  3.86612245,  3.94739229,  4.02866213,
        4.10993197,  4.19120181,  4.27247166,  4.3537415 ,  4.43501134,
        4.51628118,  4.59755102,  4.67882086,  4.7600907 ,  4.84136054,
        4.92263039,  5.00390023,  5.08517007,  5.16643991,  5.24770975,
        5.32897959,  5.41024943,  5.49151927,  5.57278912,  5.65

In [14]:
thayer_annotations_df.iloc[180:200]

,musicId,frameTime,Arousal(mean),Valence(mean)
180,5,75.5,0.161675,0.196988
181,5,76.0,0.155337,0.193662
182,5,76.5,0.157450,0.188112
183,5,77.0,0.158562,0.196662
184,5,77.5,0.155225,0.198900
185,5,78.0,0.163875,0.190337
186,5,78.5,0.159663,0.184550
187,5,79.0,0.153888,0.193087
188,5,79.5,0.159013,0.184875
189,5,80.0,0.160450,0.180113
